# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [23]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'MSTR',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [24]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (MSTR). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for MSTR to the Data Analyst. They have the expertise in statistical modeling and machine learning needed for this task.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for MSTR using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for MSTR."}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for MSTR and analyze it using statistical modeling and machine learning to predict market movements accurately.

Action: Read website content
Action Input: {"webs

Final Answer: The current market data for MSTR shows that the stock price is at $339.41, with a 0.84% increase. The company, Strategy Inc, operates as a bitcoin treasury company and offers various securities to provide investors with exposure to Bitcoin. Additionally, they provide AI-powered enterprise analytics software. The company was formerly known as MicroStrategy Incorporated and changed its name to Strategy Inc in August 2025. The market cap is $95.435B, with a beta of 3.78. The company has 1,512 full-time employees and is headquartered in Tysons Corner, Virginia. More information can be found on their website www.strategy.com.

> Finished chain.
 

The current market data for MSTR shows that the stock price is at $339.41, with a 0.84% increase. The company, Strategy Inc, operates as a bitcoin treasury company and offers various securities to provide investors with exposure to Bitcoin. Additionally, they provide AI-powered enterprise analytics software. The company was formerly 

Final Answer: 
After analyzing the latest news and updates related to MSTR on Yahoo Finance, it is observed that Strategy Inc (MSTR) is currently trading at $339.41 with a 0.84% increase. The company operates as a bitcoin treasury company and offers investors exposure to Bitcoin through securities. Additionally, MSTR provides AI-powered enterprise analytics software. The stock has shown significant growth over the past year, with a 148.67% increase. The company's market cap is $95.435 billion, and it has 1,512 full-time employees. Based on this information, MSTR presents both growth opportunities and risks in the market.

> Finished chain.
 

After analyzing the latest news and updates related to MSTR on Yahoo Finance, it is observed that Strategy Inc (MSTR) is currently trading at $339.41 with a 0.84% increase. The company operates as a bitcoin treasury company and offers investors exposure to Bitcoin through securities. Additionally, MSTR provides AI-powered enterprise analytics soft

Final Answer: Strategy Inc (MSTR) is currently trading at $339.34 with a 0.82% increase. The company operates as a bitcoin treasury company and offers exposure to Bitcoin through securities. Additionally, Strategy Inc provides AI-powered enterprise analytics software. Over the past year, MSTR has shown significant growth with a 148.67% increase. The market cap for Strategy Inc is $95.44 billion, and the company has 1,512 full-time employees. Based on the analysis, there are opportunities for growth in the market for MSTR, but there are also risks to consider. The user-defined risk tolerance for this task is Medium, and considering trading preferences for Day Trading, it is important to develop and refine trading strategies that align with these factors.

> Finished chain.
 

Strategy Inc (MSTR) is currently trading at $339.34 with a 0.82% increase. The company operates as a bitcoin treasury company and offers exposure to Bitcoin through securities. Additionally, Strategy Inc provides A

Final Answer: 
Key factors to consider when developing trading strategies for MSTR that align with the user's risk tolerance of Medium include analyzing the company's market opportunities and threats, understanding its background as a bitcoin treasury company and provider of AI-powered enterprise analytics software, and taking into account specific preferences for Day Trading. Additionally, factors such as the stock price, market cap, historical performance, industry comparisons, valuation measures, profitability, income statement, balance sheet, cash flow, and research analysis should be considered in developing effective and risk-averse trading strategies for MSTR.

> Finished chain.
 

Key factors to consider when developing trading strategies for MSTR that align with the user's risk tolerance of Medium include analyzing the company's market opportunities and threats, understanding its background as a bitcoin treasury company and provider of AI-powered enterprise analytics software,

Thought: 
I need to delegate tasks to the appropriate team members and ask relevant questions in order to gather all necessary information for the final execution plans.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Perform technical analysis for MSTR", "context": "Please analyze stock price movements, trading volume, support and resistance levels, moving averages, and other technical indicators to identify entry and exit points for trades."}

> Entering new CrewAgentExecutor chain...
I need to gather information on MSTR stock price movements, trading volume, support and resistance levels, moving averages, and other technical indicators.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/mstr"} 

marketwatch.comPlease enable JS and disable any ad blocker

Action: Search the internet
Action Input: {"search_query": "MSTR stock technical analysis"} 


Search results: Title: Techni

Thought: I should search the internet for information on MSTR's proposed trading strategies to gather insights for a comprehensive risk analysis.

Action: Search the internet
Action Input: {"search_query": "MSTR proposed trading strategies"} 


Search results: Title: Strategy: MSTR Metrics
Link: https://www.strategy.com/
Snippet: MSTR Price. $363.60. -$2.72 (-0.74%) ; 3M Return. -12% ; 1Y Return. 169% ; BSE Return. 2,841% ; Market Cap ($M). $103,173. -$772 (-0.74%).
---
Title: Strategy Falls 15% in a Month: Hold or Fold the MSTR Stock?
Link: https://finance.yahoo.com/news/strategy-falls-15-month-hold-162300910.html
Snippet: Strategy's bitcoin gains and disciplined capital raising contrast with steep share declines and premium valuation concerns.
---
Title: Deconstructing Strategy (MSTR): Premium, Leverage, and ...
Link: https://www.vaneck.com/us/en/blogs/digital-assets/matthew-sigel-deconstructing-strategy-mstr-premium-leverage-and-capital-structure/
Snippet: We analyze Strategy (MSTR)

Thought: 
I need to gather insights from my team to evaluate the risks associated with the proposed trading strategies for MSTR and provide mitigation recommendations.

Action: Delegate work to co-worker
Action Input: {"coworker": "Risk Advisor", "task": "Conduct a comprehensive risk analysis for the proposed trading strategies for MSTR and suggest mitigation strategies", "context": "Based on the fundamental and technical analysis provided, the detailed execution plans for MSTR are outlined. The Risk Advisor needs to analyze potential risks associated with the execution plan and provide detailed mitigation strategies."} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Action: Ask question to co-worker
Action Input: {"coworker": "Risk Advisor", "question": "What are the key risk factors associated with the proposed trading strategies for MSTR and what mitigation strategies would you recommend?", "context": "Based on the fundament

- Display the final result as Markdown.

In [25]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for MSTR Trading Strategies:

After analyzing the proposed trading strategies for MSTR and considering the potential risks associated with them, several key risk factors have been identified. These include the risk of price drops, the impact of the collapse of financing Bitcoin on premium contraction, potential systemic risks, unintended consequences that regulators should proactively mitigate, and the disclosure of 64 risk factors in the most recent earnings report, particularly in the "Finance & Corporate" category. Additionally, external factors such as the Trump administration's pro-crypto stance, discussions around potential U.S. strategic decisions, and the unregulated nature and lack of transparency in operations pose significant risks.

To mitigate these risks, it is recommended to closely monitor price movements, diversify the portfolio to reduce exposure to any single risk factor, conduct thorough research and due diligence, stay informed on regulatory developments, implement risk management strategies like stop-loss orders, and maintain a long-term investment perspective to withstand market fluctuations and potential downturns. By taking these steps, the team can effectively manage risks and capitalize on market opportunities while executing trades for MSTR.